# Data triming and filtering with fastp

[fastp](https://github.com/OpenGene/fastp) is "A tool designed to provide fast all-in-one preprocessing for FastQ files. This tool is developed in C++ with multithreading supported to afford high performance."

In otherwords, we can take the FASTQ files (our sequencing data) and do various things to improve the quality of that data such as

- Remove reads that have an overal low quality (e.g., low [Phred score](https://en.wikipedia.org/wiki/Phred_quality_score))
- Trim portions of a read (e.g., where the first few/last few bases are of low quality)
- Remove reads that are shorter than some desired length (e.g., drop short reads and keep long reads)
- Remove adapters (e.g., in sequencing, some artificial DNA "tags" may be added to the DNA to be sequenced)
- And more...

## Installing fastp

As always, we will install the software; again we will use conda. 

**Important**: Make sure you execute each numbered step 

1. We will search for the tool we want to install

We will use the `conda search` command and the channel (`-c`) flag to search [bioconda](https://bioconda.github.io/)

In [ ]:
conda search fastp -c bioconda

2. Create a conda enviornment

Conda uses something called "enviornments" which are essentially isolated configurations on our computer where we can included all the needed compatible tools and exlude other tools which are unnesessary or would have conflicts with our desired tool. We will use the `-y` option to install without prompting the user for input, the `--name` option to name the enviornment for the tool. We will enforce versioning (`tool==version`) so that we know what version of a tool was used to do an analysis should we wish to repeat the analysis. 

**Tip**: Use the latest version where possible, but if you get an error with dependancies, using a lower version may help. Some tools may never be installed successfully using conda, but we will face those when we have too. 


In [ ]:
conda create -y --name fastp fastp==0.20.0 -c bioconda

3. We will use the `conda init` command so that conda can be configured for this shell

In [1]:
conda init

no change     /opt/anaconda3/condabin/conda
no change     /opt/anaconda3/bin/conda
no change     /opt/anaconda3/bin/conda-env
no change     /opt/anaconda3/bin/activate
no change     /opt/anaconda3/bin/deactivate
no change     /opt/anaconda3/etc/profile.d/conda.sh
no change     /opt/anaconda3/etc/fish/conf.d/conda.fish
no change     /opt/anaconda3/shell/condabin/Conda.psm1
no change     /opt/anaconda3/shell/condabin/conda-hook.ps1
no change     /opt/anaconda3/lib/python3.9/site-packages/xontrib/conda.xsh
no change     /opt/anaconda3/etc/profile.d/conda.csh
no change     /home/exouser/.bashrc
No action taken.


4. **DON'T SKIP**: We need to restart the computer's [kernal](https://en.wikipedia.org/wiki/Kernel_(operating_system)). Go to the **Kernal** menu and choose **Restart Kernal**

5. Finally, we can activate the conda enviornment (created with the name used for the environment). When you run the next cell it should return the name of the environment.  

In [2]:
conda activate fastp

(fastp) 

: 1

## Connecting to our project and dataset folders

We have to check that this notebook is running in the expected directory, and that you have a project folder. When you run the following command, your working directory should be `/home/exouser`

1. Check our working directory

In [ ]:
pwd

**(Optional)**: If the `pwd` command did not give you `/home/exouser` run the following command, otherwise **skip**. Please also keep in mind - you **should not** be running this notebook in `/mnt/ceph/notebooks`, so ask for help if you get this result.

In [ ]:
cd /home/exouser # only run this if needed

2. Inspect the `project` folder. 

You should also have a project folder which is linked to your home folder on the shared drive. Running the following command should generate something like:

`lrwxrwxrwx 1 exouser exouser 19 Dec 30 21:27 jwilliams -> /mnt/ceph/YOURUSERNAME`

In [ ]:
ls -lh project --color=never

**(Optional)**: If you don't have a `project` folder linked to your folder on the shared drive, run the command below. Remeber to change `YOURUSERNAME` to the name of your folder. 

In [ ]:
mkdir -p project && ln -s /mnt/ceph/YOURUSERNAME project # run only if needed, fix YOURUSERNAME

3. Connect to and inspect the folder of FASTQ reads. 

Let's also make sure the project folder has a link to the FASTQ reads we will use and also create a place for the results of your FastQC analysis. 

The next command will create a link in your project folder to a location on the shared drive where we have placed all of the Chamecrista FASTQ reads. 

In [ ]:
ln -s /mnt/ceph/chamecrista_sup_fastq project/chamecrista_fastq

The following summarizes what we have:

- `called_reads.fastq` : All of the reads done during the 2022 sequencing project
- `100_called_reads.fastq`: A subset of the first 100 reads in `called_reads.fastq`
- `1000_called_reads.fastq`: A subset of the first 1000 reads in `called_reads.fastq`
- `10000_called_reads.fastq`: A subset of the first 10000 reads in `called_reads.fastq`

The versions of the above withe the `.gz` file extension are compressed versions of the above list. 

5. Create an output folder for the `fastp` analysis. 

Finally, let's create a place in our project directory to store the output of our `fastp` analysis. 

In [8]:
mkdir -p ~/project/fastp_results

(fastp) 

: 1

## Running fastp

First, we will be run fastp on a small subset of the sequence reads. In this exercise, the reads are in the `project/chamecrista_fastq` folder. We will run on a sample that has 1000 fastq files from the experiment (`1000_called_reads.fastq.gz`). 

**Tip**: When using commands or searching for files, the tab key will help you autocomplete (and help ensure the files and commands you think you have are actually accessible).



### fastp options

When we call the `fastp` program here are some options we are likely to use (see the [fastp](https://github.com/OpenGene/fastp) documentation for all options):

**Note**: We are working with "single-end" data so we will only cover these options

- `--in1 <file_name.fastq.gz>`: Name of the input file (this is the file to be trimmed)
- `--out1 <file_name.fastq.gz>`: Name of the output FASTQ file that should be created (i.e. whatever read triming and filtering; you get to name the output file, choose something descriptive like "trimmed"
- `--length_required  <some number>`: Remove reads less than this length (bp)
- `--average_qual <some number>`: Remove reads when their average quality (Phred score) is below some number. Don't be too strict with Nanopore reads since we expect a lower Phred score on average. Reads with an average less than 9 have been automaticaly excluded during the initial sequencing. 
- `--thread <some number>`: How many CPUs to use (more will be faster, but you can only use with as many CPUs as you have launched this application with). 
- `--report_title "some title in quotes`: A descriptive title for your summary report
- `--html <filename.html>`: The name of the file to save your HTML report in

### Example 1 - running fastp on `1000_called_reads.fastq.gz`

1. Review and run the following `fastp` analysis. Ensure you have a sense of what this command should do

**Tip**: Adding a backslash after each command does not change the commands, it just makes it easier to read. These symbols are not interprited by the computer. 

In [9]:
fastp --in1 /home/exouser/project/chamecrista_fastq/1000_called_reads.fastq.gz\
      --out1 /home/exouser/project/fastp_results/1000_called_reads_filtered.fastq.gz\
      --length_required 1000\
      --thread 16\
      --report_title "1000 Reads Sample Reads>= 1000bp"\
      --html /home/exouser/project/fastp_results/1000_called_reads_filtered_example.html\
      --json /home/exouser/project/fastp_results/1000_called_reads_filtered_example.json

Detecting adapter sequence for read1...
No adapter detected for read1

Read1 before filtering:
total reads: 1000
total bases: 3485276
Q20 bases: 2986486(85.6887%)
Q30 bases: 2445108(70.1554%)

Read1 after filtering:
total reads: 567
total bases: 3260015
Q20 bases: 2809693(86.1865%)
Q30 bases: 2302344(70.6237%)

Filtering result:
reads passed filter: 567
reads failed due to low quality: 15
reads failed due to too many N: 0
reads failed due to too short: 418
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 0%

JSON report: /home/exouser/project/fastp_results/1000_called_reads_filtered_example.json
HTML report: /home/exouser/project/fastp_results/1000_called_reads_filtered_example.html

fastp --in1 /home/exouser/project/chamecrista_fastq/1000_called_reads.fastq.gz --out1 /home/exouser/project/fastp_results/1000_called_reads_filtered.fastq.gz --length_required 1000 --thread 16 --report_title 1000 Reads Sample Rea

: 1

2. In your Jupyter file browser, navigate to `~/project/fastp_results`; you should now have an HTML file entitled `1000_called_reads_filtered_example.html`. Double-click to open and examine the file. 
**Tip**: This file contains graphcs, but you must click the **Trust HTML** button that will be in the upper-left of the tab that opens

### Results

Clearly, filtering only by length the number of total reads changes (from 1000 to only 567 that passed this filter). In Nanopore sequencing, length is not really related to sequencing quality. So the other statistics will change (i.e. the number of bases with a Phred score of at least 20), but these changes are minor/not significant. 

## Challenge - Use FastP to create your input sequences for assembly

Now, it's up to you to take the entire sequencing dataset (located in `~/project/chamecrista_fastq/called_reads.fastq.gz`) and make choices about how much you want to clean and filter this data. The better data you present to the assembly software, the better the genome assembly you will get. However, there are tradeoffs. 

Think of this as if you were making an apple pie. You go to by apples, but some have brown spots and some are spoiled. No one wants a pie made of spoiled apples. However, if you insist only on 100% perfect apples, there may not be enough for your pie. 

In your genome assembly, having longer, high quality reads will help. However you need a minimum of 30-40X   [coverage](https://en.wikipedia.org/wiki/Coverage_(genetics)) for most assemblers to work. 


### What to do

1. Try some fastp filter/trimming tools on the `called_reads.fastq.gz` file located at `~/project/chamecrista_fastq/called_reads.fastq.gz`

**Your command should**
- start with `fastp`
- specify the `--in1` and `--out1` parameters (give your output file a unique and descriptive name)
- Use the `--thread` and set to the number of CPUs used when you launched this application on CyVerse
- specify the `--report_title` and `--html` and `--json` parameters (again use unique and descriptive names/titles)

2. Take note of the number of reads you start and end with - don't be too strict
3. You can combine as several of the triming and flitering tools in the [fastp](https://github.com/OpenGene/fastp) documentation
4. You should create at least two outputs of reads for triming and filtering - perhaps one that is very conservative (higher read lengths, base qualities) and one not so conservative. 

In the end, the only way you will know which of your choices are better is to run the genome assembler and see what results you get. Different assemblers will be tolerant of different lengths and qualities. Some assemblers may actually do better with large numbers of lower quality reads (because they use the sheer number of reads to correct for errors).

In [ ]:
# Add as many cells as you need to run your commands. 
# Make sure you use unique file names and reports each time to keep track. 
# Be sure to save your work to your folder on CyVerse 
# You can save to `data/output/` as above or 
# `/home/shared/cyverse_training/classrooms/dnalc_genome_camp/student_work/YOURFOLDERNAME`


## Document your work

This is the first notebook where you will be manipulating a file. We need to keep good track on what changes were made/how a file was produced so that we can fully document our work. In this exercise, it will be critical to know how you trimmed and filtered reads. Knowing your settings will allow us to compare results across everyone who does this experiment. You will also be able to go back and reproduce your work if needed. 
